In [ ]:
import tkinter as tk
import time
import csv
import os

PROMPTS = [
    "the quick brown fox jumps over the lazy dog",
    "hello world",
    "pack my box with five dozen liquor jugs",
]

DATA_DIR = "typing_data"
os.makedirs(DATA_DIR, exist_ok=True)

CSV_FILE = os.path.join(DATA_DIR, "keystroke_data.csv")

# Write CSV header if file doesn't exist
if not os.path.isfile(CSV_FILE):
    with open(CSV_FILE, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "valid", "timestamp", "event_type", "key"
        ])

class TypingBiometricsApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Typing Biometrics Capture")

        # Session state
        self.prompts = PROMPTS.copy()
        self.current_index = 0
        self.current_prompt = None
        self.events = []
        self.session_active = False

        # Valid/Invalid selection
        tk.Label(root, text="Valid or Invalid?").pack()
        self.valid = tk.StringVar(value="1")
        tk.Radiobutton(root, text="Valid", variable=self.valid, value="1").pack()
        tk.Radiobutton(root, text="Invalid", variable=self.valid, value="0").pack()

        # Prompt label
        self.prompt_label = tk.Label(root, text="", font=("Helvetica", 14), pady=10)
        self.prompt_label.pack()

        # Feedback label
        self.feedback_label = tk.Label(root, text="", font=("Helvetica", 12))
        self.feedback_label.pack()

        # Typing box
        self.textbox = tk.Text(root, height=3, width=60, font=("Consolas", 12))
        self.textbox.pack()
        self.textbox.bind("<KeyPress>", self.on_key_down)
        self.textbox.bind("<KeyRelease>", self.on_key_up)

        # Start first prompt automatically
        self.start_session()

    def start_session(self):
        self.current_prompt = self.prompts[self.current_index]
        self.prompt_label.config(text=f"Please type this:\n\"{self.current_prompt}\"")
        self.feedback_label.config(text="", fg="black")
        self.textbox.delete("1.0", tk.END)
        self.textbox.focus_set()
        self.events = []
        self.session_active = True

    def on_key_down(self, event):
        if not self.session_active:
            return
        if event.keysym == "Return":
            self.validate_input()
            return
        self.record_event(event, "down")

    def on_key_up(self, event):
        if not self.session_active:
            return
        if event.keysym == "Return":
            return
        self.record_event(event, "up")

    def record_event(self, event, event_type):
        timestamp = time.time()
        self.events.append({
            "valid": self.valid.get(),
            "prompt": self.current_prompt,
            "timestamp": timestamp,
            "event_type": event_type,
            "key": event.keysym,
        })

    def validate_input(self):
        typed = self.textbox.get("1.0", tk.END).strip()
        success = typed == self.current_prompt

        if not success:
            self.feedback_label.config(text="✗ Try Again", fg="red")
            self.events.clear()
            self.textbox.focus_set()
            return

        self.feedback_label.config(text="✓ Accepted", fg="green")
        self.save_session()
        self.session_active = False

        # Automatically go to next prompt after short delay
        self.root.after(500, self.next_prompt)

    def save_session(self):
        with open(CSV_FILE, mode="a", newline="") as f:
            writer = csv.writer(f)
            for e in self.events:
                writer.writerow([
                    e["valid"],
                    f"{e['timestamp']:.6f}",
                    e["event_type"],
                    e["key"],
                ])
        self.events.clear()

    def next_prompt(self):
        self.current_index = (self.current_index + 1) % len(self.prompts)
        self.start_session()

if __name__ == "__main__":
    root = tk.Tk()
    app = TypingBiometricsApp(root)
    root.mainloop()
